# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
df = pd.read_csv("weather_data.csv")
df.info()
df = df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         100 non-null    object 
 1   Country      87 non-null     object 
 2   Latitude     87 non-null     float64
 3   Longitude    87 non-null     float64
 4   Date         87 non-null     float64
 5   Wind_Speed   87 non-null     float64
 6   Pressure     87 non-null     float64
 7   Humidity     87 non-null     float64
 8   Temperature  87 non-null     float64
 9   Cloudiness   87 non-null     float64
dtypes: float64(8), object(2)
memory usage: 7.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         87 non-null     object 
 1   Country      87 non-null     object 
 2   Latitude     87 non-null     float64
 3   Longitude    87 non-nu

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
locations = df.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
sub = (df.Temperature < 80) & (df.Temperature > 70) & (df.Wind_Speed < 10) & (df.Cloudiness == 0)
df2 = df.loc[sub]
df2

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
11,Zhaoqing,CN,23.0512,112.4597,1.668101e+09,0.81,1015.0,81.0,76.82,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
coord = f"{23.0512}, {112.4597}"
radius = 5000
hotel = "lodging"


parameters = {
    "location": coord,
    "radius": radius,
    "type": hotel,
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=parameters)
data = response.json()
print(data["results"])

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 23.092145, 'lng': 112.474735}, 'viewport': {'northeast': {'lat': 23.0934285802915, 'lng': 112.4759329802915}, 'southwest': {'lat': 23.0907306197085, 'lng': 112.4732350197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': '七星岩旅游度假区', 'photos': [{'height': 2322, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114400738587649543554">HsienCheng Liu</a>'], 'photo_reference': 'AW30NDzbts9cd5pVrjIAC5aPEqnd6bp6Wi1wJrAdgUht_97is6gwFxszlEGzP8wPDQdsBIhAhJQfQVzw7Od95WX4kfwHF_Juv-5WwKNCmj-s7-K3lODaY9OVShujOPSoRMymKM3MeOMjbL8h7JIWoCahl0RJB4rrta4ZgOMAGytx6LHHdgp8', 'width': 4128}], 'place_id': 'ChIJgUyFvKnRqTYRPJszrVLs8hc', 'plus_code': {'compound_code': '3FRF+VV Duanzhou District, Zhaoqing, Guangdong Province, China', 'global_code': '7P

In [21]:
Hotels = []
Ratings = []
Location = []
Cities = []
Countries = []
Lat = []
Long = []

for _, row in df2.iterrows():
    print(row.City)

    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=parameters)
    data = response.json()
    hotel = data["results"][0]["name"]
    address = data["results"][0]["vicinity"]
    rating = data["results"][0]["rating"]
    
    # put it all in a list
    Hotels.append(hotel)
    Ratings.append(rating)
    Location.append(address)
    Cities.append(row.City)
    Countries.append(row.Country)
    Lat.append(row.Latitude)
    Long.append(row.Longitude)

Zhaoqing


In [23]:
hoteldf = pd.DataFrame()
hoteldf['City'] = Cities
hoteldf['Country'] = Countries
hoteldf['Latitude'] = Lat
hoteldf['Longitude'] = Long
hoteldf['Hotel'] = Hotels
hoteldf['Address'] = Location
hoteldf['Rating'] = Ratings

hoteldf

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Zhaoqing,CN,23.0512,112.4597,七星岩旅游度假区,China,4.4


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
locations = hoteldf[["Latitude", "Longitude"]]

In [31]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=20, point_radius=5
)


fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)


fig

Figure(layout=FigureLayout(height='420px'))